## Part 1
-average age of the listener 
-percentage of plays that came from a user who played the song exactly once

-percentage of plays that came from a user who played the song at least five times

If there any missing values, decide how you will deal with them, and summarize your approach.

## Part 2
data.describe()
ice.mean() 

In this same pre-processing notebook, explore the data by generating some descriptive statistics. After each code block, you should have summary statements, or code blocks should print out summary statements that are useful to a reader.

## Part 3

Complete this notebook by exporting the generated dataframe using pickle for posterity, but don't commit your pickled file to the repository. Sometimes, it is more convenient to process the data locally rather than transmitting large processed data files.

import pandas as pd
ice = pd.read_csv('SeaIce.txt', delim_whitespace = True)
print ('shape:', ice.shape)
ice.head() 

import pickle
pickle_in = open("data.pkl", "rb")
SeaIce = pickle.load(pickle_in)
print (type(SeaIce))

x = SeaIce.year
y = SeaIce.extent
plt.scatter(x, y, color = 'red')
plt.xlabel('Month')
plt.ylabel('Extent')

## Part 4

In a new notebook, load the dataframe object. Using k-fold cross-validation, apply a linear regression model. Here, the response variable is the number of times a song has been played based on what we know about the listeners. Be sure to return the model score on both the training and validation data.

## Part 5

Explain your results. Specifically address, if the assumption of linear response to the input features was reasonable. You will want to study a plot of the model's residual. Ideally, the residuals are small in magnitude, and are evenly distributed about the true values. What are some possible improvements that can be made?

## Part 6

Suppose we are instead curious about whether a song will be played by a user based on a demographics. Apply a logistic regression model and report your results.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

userdat = pd.read_csv('data/lastfm-dataset-1K/userid-profile.tsv', sep = '\t',header =0,error_bad_lines=False)
songsdat = pd.read_csv('data/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', sep = '\t', header = None,error_bad_lines=False)
songsdat.columns = ['#id','timestamp','artid','artname','traid','traname']



Skipping line 2120260: expected 6 fields, saw 8

Skipping line 2446318: expected 6 fields, saw 8

Skipping line 11141081: expected 6 fields, saw 8

Skipping line 11152099: expected 6 fields, saw 12
Skipping line 11152402: expected 6 fields, saw 8

Skipping line 11882087: expected 6 fields, saw 8

Skipping line 12902539: expected 6 fields, saw 8
Skipping line 12935044: expected 6 fields, saw 8



In [9]:
userdat.head()

#id gender   age        country    registered
0  user_000001      m   NaN          Japan  Aug 13, 2006
1  user_000002      f   NaN           Peru  Feb 24, 2006
2  user_000003      m  22.0  United States  Oct 30, 2005
3  user_000004      f   NaN            NaN  Apr 26, 2006
4  user_000005      m   NaN       Bulgaria  Jun 29, 2006

In [10]:
songsdat.head()

#id             timestamp                                 artid  \
0  user_000001  2009-05-04T23:08:57Z  f1b1cf71-bd35-4e99-8624-24a6e15f133a   
1  user_000001  2009-05-04T13:54:10Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   
2  user_000001  2009-05-04T13:52:04Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   
3  user_000001  2009-05-04T13:42:52Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   
4  user_000001  2009-05-04T13:42:11Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   

     artname traid                                     traname  
0  Deep Dish   NaN  Fuck Me Im Famous (Pacha Ibiza)-09-28-2007  
1       坂本龍一   NaN           Composition 0919 (Live_2009_4_15)  
2       坂本龍一   NaN                        Mc2 (Live_2009_4_15)  
3       坂本龍一   NaN                     Hibari (Live_2009_4_15)  
4       坂本龍一   NaN                        Mc1 (Live_2009_4_15)

In [4]:
music=songsdat.merge(userdat,left_on='#id', right_on='#id', how='inner')
music.head()

#id             timestamp                                 artid  \
0  user_000001  2009-05-04T23:08:57Z  f1b1cf71-bd35-4e99-8624-24a6e15f133a   
1  user_000001  2009-05-04T13:54:10Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   
2  user_000001  2009-05-04T13:52:04Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   
3  user_000001  2009-05-04T13:42:52Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   
4  user_000001  2009-05-04T13:42:11Z  a7f7df4a-77d8-4f12-8acd-5c60c93f4de8   

     artname traid                                     traname gender  age  \
0  Deep Dish   NaN  Fuck Me Im Famous (Pacha Ibiza)-09-28-2007      m  NaN   
1       坂本龍一   NaN           Composition 0919 (Live_2009_4_15)      m  NaN   
2       坂本龍一   NaN                        Mc2 (Live_2009_4_15)      m  NaN   
3       坂本龍一   NaN                     Hibari (Live_2009_4_15)      m  NaN   
4       坂本龍一   NaN                        Mc1 (Live_2009_4_15)      m  NaN   

  country    registered  
0   Japan  Aug 13, 2006  
1   Japan  Aug 13, 2006  
2   Japan  Aug 13, 2006  
3   Japan  Aug 13, 2006  
4   Japan  Aug 13, 2006

In [4]:
print('shape:', music.shape)

('shape:', (15051053, 10))


In [5]:
# calculating the number of times each song has been played
vct = music['traname'].value_counts()
vct

Intro                                                            14329
All I Need                                                        5131
Home                                                              4873
[Untitled]                                                        4732
Untitled                                                          4720
Crazy                                                             4626
Angel                                                             4553
Such Great Heights                                                4268
Breathe                                                           4252
One                                                               4217
Wake Up                                                           4146
Love Will Tear Us Apart                                           4082
Hallelujah                                                        3979
Lullaby                                                           3891
Closer

In [72]:
type(vct)
vct=vct.reset_index(level=[0])
vct.shape

ValueError: cannot insert level_0, already exists

In [6]:
musicndup = music.drop_duplicates(subset=['#id','traname'])
musicndup

#id             timestamp  \
0         user_000001  2009-05-04T23:08:57Z   
1         user_000001  2009-05-04T13:54:10Z   
2         user_000001  2009-05-04T13:52:04Z   
3         user_000001  2009-05-04T13:42:52Z   
4         user_000001  2009-05-04T13:42:11Z   
5         user_000001  2009-05-04T13:38:31Z   
6         user_000001  2009-05-04T13:33:28Z   
7         user_000001  2009-05-04T13:23:45Z   
8         user_000001  2009-05-04T13:19:22Z   
9         user_000001  2009-05-04T13:13:38Z   
10        user_000001  2009-05-04T13:06:09Z   
11        user_000001  2009-05-04T13:00:48Z   
12        user_000001  2009-05-04T12:55:34Z   
13        user_000001  2009-05-04T12:51:26Z   
14        user_000001  2009-05-03T15:48:25Z   
15        user_000001  2009-05-03T15:37:56Z   
16        user_000001  2009-05-03T15:14:53Z   
17        user_000001  2009-05-03T15:10:18Z   
18        user_000001  2009-05-03T15:04:31Z   
19        user_000001  2009-05-03T14:56:25Z   
20        user_000001  2009-05-03T14:50:51Z   
21        user_000001  2009-05-03T14:46:29Z   
22        user_000001  2009-05-03T14:39:20Z   
23        user_000001  2009-05-03T14:34:06Z   
24        user_000001  2009-05-03T14:28:02Z   
25        user_000001  2009-05-03T14:20:15Z   
26        user_000001  2009-05-03T14:07:40Z   
27        user_000001  2009-05-03T14:01:15Z   
28        user_000001  2009-05-03T13:56:25Z   
29        user_000001  2009-05-03T13:47:20Z   
...               ...                   ...   
15051023  user_001000  2008-01-28T00:13:51Z   
15051024  user_001000  2008-01-28T00:09:09Z   
15051025  user_001000  2008-01-28T00:05:19Z   
15051026  user_001000  2008-01-28T00:01:32Z   
15051027  user_001000  2008-01-27T23:26:56Z   
15051028  user_001000  2008-01-27T23:23:26Z   
15051029  user_001000  2008-01-27T23:16:30Z   
15051030  user_001000  2008-01-27T23:13:33Z   
15051031  user_001000  2008-01-27T23:09:37Z   
15051032  user_001000  2008-01-27T23:05:59Z   
15051033  user_001000  2008-01-27T23:02:12Z   
15051034  user_001000  2008-01-27T22:56:38Z   
15051035  user_001000  2008-01-27T22:52:47Z   
15051036  user_001000  2008-01-27T22:49:21Z   
15051037  user_001000  2008-01-27T22:46:00Z   
15051038  user_001000  2008-01-27T22:42:19Z   
15051039  user_001000  2008-01-27T22:39:34Z   
15051040  user_001000  2008-01-27T22:35:15Z   
15051041  user_001000  2008-01-27T22:30:32Z   
15051042  user_001000  2008-01-27T22:26:45Z   
15051043  user_001000  2008-01-27T22:22:43Z   
15051044  user_001000  2008-01-27T22:19:07Z   
15051045  user_001000  2008-01-27T22:14:39Z   
15051046  user_001000  2008-01-27T22:10:28Z   
15051047  user_001000  2008-01-27T22:05:54Z   
15051048  user_001000  2008-01-27T22:02:35Z   
15051049  user_001000  2008-01-27T21:56:52Z   
15051050  user_001000  2008-01-27T21:52:36Z   
15051051  user_001000  2008-01-27T21:49:12Z   
15051052  user_001000  2008-01-27T21:43:14Z   

                                         artid          artname  \
0         f1b1cf71-bd35-4e99-8624-24a6e15f133a        Deep Dish   
1         a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
2         a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
3         a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
4         a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
5         a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
6         a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
7         a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
8         a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
9         a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
10        a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
11        a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
12        a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
13        a7f7df4a-77d8-4f12-8acd-5c60c93f4de8             坂本龍一   
14        ba2f4f3b-0293-4bc8-bb94-2f73b5207343       Underworld   
15        ba2f4f3b-0293-4bc8-bb

In [7]:
musicndup[musicndup['gender']=="m"]['traname'].value_counts(normalize=True)*100

Intro                                                         0.016020
Breathe                                                       0.011775
One                                                           0.011522
Angel                                                         0.011472
Untitled                                                      0.011067
Wake Up                                                       0.010966
Home                                                          0.010714
Time                                                          0.010714
Fire                                                          0.009855
Hold On                                                       0.009855
Crazy                                                         0.009703
Lullaby                                                       0.009703
Closer                                                        0.009652
Smile                                                         0.009602
Foreve

In [8]:
vct1 = (music['traname'].value_counts() == 1)
vct1.describe()

count     940755
unique         2
top        False
freq      628327
Name: traname, dtype: object

In [10]:
vct5 = (music['traname'].value_counts() >= 5)
vct5.describe()

count     940755
unique         2
top        False
freq      584356
Name: traname, dtype: object

In [22]:
tsinfo = music['timestamp'].describe()
ageinfo = music['age'].describe()

music.info()
tsinfo
ageinfo

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15051053 entries, 0 to 15051052
Data columns (total 10 columns):
#id           object
timestamp     object
artid         object
artname       object
traid         object
traname       object
gender        object
age           float64
country       object
registered    object
dtypes: float64(1), object(9)
memory usage: 1.2+ GB


count    5.629318e+06
mean     2.523376e+01
std      6.446155e+00
min      3.000000e+00
25%      2.100000e+01
50%      2.400000e+01
75%      2.800000e+01
max      1.030000e+02
Name: age, dtype: float64

In [11]:
music.age.describe()

count    5.629318e+06
mean     2.523376e+01
std      6.446155e+00
min      3.000000e+00
25%      2.100000e+01
50%      2.400000e+01
75%      2.800000e+01
max      1.030000e+02
Name: age, dtype: float64

In [12]:
import pickle
pickle_out = open('data.pkl', 'wb')
pickle.dump(music, pickle_out)
# Pickle the 'data' dictionary using the highest protocol available.
pickle.dump(music2, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()

IOError: [Errno 122] Disk quota exceeded